In [8]:
import cv2
import numpy as np
import math

def calculate_angle(p1, p2):
    """
    2点間の角度を計算 (単位: 度)
    """
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    angle = math.atan2(dy, dx) * (180 / math.pi)  # ラジアンを度に変換
    return angle

def main():
    # マスク画像を読み込み（白黒画像として）
    mask_img = cv2.imread('images/background/parts/back_part4-2_mask.png', cv2.IMREAD_GRAYSCALE)

    # 二値化処理（輪郭抽出のために必要）
    _, binary_img = cv2.threshold(mask_img, 127, 255, cv2.THRESH_BINARY)

    # 輪郭を検出
    contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 最初の輪郭を選択（例として最初のものだけを扱う）
    if len(contours) == 0:
        print("輪郭が見つかりませんでした。")
        return

    contour = contours[0]  # 最初の輪郭

    # 輪郭の各セグメントの角度を計算
    angles = []
    for i in range(len(contour)):
        p1 = contour[i][0]  # 現在の点
        p2 = contour[(i + 1) % len(contour)][0]  # 次の点（ループ処理）
        angle = calculate_angle(p1, p2)
        angles.append(angle)
        print(f"セグメント {i}: 点 {p1} -> 点 {p2}, 角度: {angle:.2f} 度")

    # 結果を画像に描画
    output_img = cv2.cvtColor(binary_img, cv2.COLOR_GRAY2BGR)
    for i in range(len(contour)):
        p1 = tuple(contour[i][0])
        p2 = tuple(contour[(i + 1) % len(contour)][0])
        cv2.line(output_img, p1, p2, (0, 255, 0), 2)  # 緑色の線でセグメントを描画

        # 角度を表示
        angle_text = f"{angles[i]:.1f}°"
        mid_point = ((p1[0] + p2[0]) // 2, (p1[1] + p2[1]) // 2)
        cv2.putText(output_img, angle_text, mid_point, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    # 結果を保存・表示
    cv2.imwrite('contour_angles_output.png', output_img)
    cv2.imshow('Contour Angles', output_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


セグメント 0: 点 [372  27] -> 点 [371  28], 角度: 135.00 度
セグメント 1: 点 [371  28] -> 点 [367  28], 角度: 180.00 度
セグメント 2: 点 [367  28] -> 点 [366  29], 角度: 135.00 度
セグメント 3: 点 [366  29] -> 点 [363  29], 角度: 180.00 度
セグメント 4: 点 [363  29] -> 点 [362  30], 角度: 135.00 度
セグメント 5: 点 [362  30] -> 点 [360  30], 角度: 180.00 度
セグメント 6: 点 [360  30] -> 点 [359  31], 角度: 135.00 度
セグメント 7: 点 [359  31] -> 点 [357  31], 角度: 180.00 度
セグメント 8: 点 [357  31] -> 点 [356  32], 角度: 135.00 度
セグメント 9: 点 [356  32] -> 点 [354  32], 角度: 180.00 度
セグメント 10: 点 [354  32] -> 点 [353  33], 角度: 135.00 度
セグメント 11: 点 [353  33] -> 点 [352  33], 角度: 180.00 度
セグメント 12: 点 [352  33] -> 点 [351  34], 角度: 135.00 度
セグメント 13: 点 [351  34] -> 点 [348  34], 角度: 180.00 度
セグメント 14: 点 [348  34] -> 点 [347  35], 角度: 135.00 度
セグメント 15: 点 [347  35] -> 点 [331  35], 角度: 180.00 度
セグメント 16: 点 [331  35] -> 点 [330  34], 角度: -135.00 度
セグメント 17: 点 [330  34] -> 点 [320  34], 角度: 180.00 度
セグメント 18: 点 [320  34] -> 点 [320 212], 角度: 90.00 度
セグメント 19: 点 [320 212] -> 点 [448 212], 角度:

In [10]:
import cv2
import numpy as np
import math

def calculate_angle_from_contour(image_path, point1_index=0, point2_offset=10):
    # 1. 画像の読み込み
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError("指定された画像が見つかりません")

    # 2. 二値化処理
    _, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

    # 3. 輪郭の検出
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        raise ValueError("輪郭が検出されませんでした")

    # 最初の輪郭を使用
    contour = contours[0]

    # 4. 輪郭の点を2つ選択 (間隔を point2_offset で調整)
    if len(contour) < 2:
        raise ValueError("輪郭の点が不足しています")

    point1 = contour[point1_index][0]
    point2_index = (point1_index + point2_offset) % len(contour)  # 周回計算で範囲外アクセスを防ぐ
    point2 = contour[point2_index][0]

    # 5. 距離を計算して角度を取得
    dx = point2[0] - point1[0]
    dy = point2[1] - point1[1]
    angle_radians = math.atan2(dy, dx)  # ラジアン単位
    angle_degrees = math.degrees(angle_radians)  # 度単位に変換

    print(f"Point1: {point1}, Point2: {point2}")
    print(f"dx: {dx}, dy: {dy}")
    print(f"Angle (radians): {angle_radians}, Angle (degrees): {angle_degrees}")

    return angle_radians, angle_degrees

# 画像パスを指定
image_path = "images/background/parts/back_part4-2_mask.png"

# 関数を呼び出して角度を取得
try:
    angle_radians, angle_degrees = calculate_angle_from_contour(image_path, point1_index=0, point2_offset=20)
except Exception as e:
    print(e)


Point1: [372  27], Point2: [448 212]
dx: 76, dy: 185
Angle (radians): 1.1810051654290934, Angle (degrees): 67.66661156223665


In [47]:
import cv2
import numpy as np
import math

def calculate_angle_from_contour(image_path, point1_index=0, point2_offset=10):
    # 1. 画像の読み込み
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError("指定された画像が見つかりません")

    # 2. 二値化処理
    _, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

    # 3. 輪郭の検出
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        raise ValueError("輪郭が検出されませんでした")

    # 最初の輪郭を使用
    contour = contours[0]

    # 4. 輪郭の点を2つ選択 (間隔を point2_offset で調整)
    if len(contour) < 2:
        raise ValueError("輪郭の点が不足しています")

    point1 = contour[point1_index][0]
    point2_index = (point1_index + point2_offset) % len(contour)  # 周回計算で範囲外アクセスを防ぐ
    point2 = contour[point2_index][0]

    # 5. 距離を計算して角度を取得
    dx = point2[0] - point1[0]
    dy = point2[1] - point1[1]
    angle_radians = math.atan2(dy, dx)  # ラジアン単位
    angle_degrees = math.degrees(angle_radians)  # 度単位に変換

    print(f"Point1: {point1}, Point2: {point2}")
    print(f"dx: {dx}, dy: {dy}")
    print(f"Angle (radians): {angle_radians}, Angle (degrees): {angle_degrees}")

    # 6. 結果を画像に描画して表示
    img_color = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)  # 表示用カラー画像
    cv2.line(img_color, tuple(point1), tuple(point2), (0, 255, 0), 2)  # 線を描画
    cv2.circle(img_color, tuple(point1), 5, (255, 0, 0), -1)  # Point1に青い円を描画
    cv2.circle(img_color, tuple(point2), 5, (0, 0, 255), -1)  # Point2に赤い円を描画
    text_position = (10, 30)  # 角度表示位置
    cv2.putText(
        img_color,
        f"Angle: {angle_degrees:.2f} ",
        text_position,
        cv2.FONT_HERSHEY_SIMPLEX,
        0.8,
        (0, 255, 255),
        2,
        cv2.LINE_AA,
    )

    # 画像を表示
    cv2.imshow("Contour Angle", img_color)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return angle_radians, angle_degrees

# 画像パスを指定
image_path = "images/background/wakuta_mask.png"  # 画像パスを適宜変更

# 関数を呼び出して角度を取得
try:
    angle_radians, angle_degrees = calculate_angle_from_contour(image_path, point1_index=0, point2_offset=10)
except Exception as e:
    print(e)


Point1: [370 123], Point2: [341 128]
dx: -29, dy: 5
Angle (radians): 2.9708574421145104, Angle (degrees): 170.21759296819272


In [40]:
import cv2
import numpy as np
import math

def calculate_angle_from_contour(image_path, point1_index=0, point2_offset=10):
    # 1. 画像の読み込み
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError("指定された画像が見つかりません")

    # 2. 二値化処理
    _, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

    # 3. 輪郭の検出
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        raise ValueError("輪郭が検出されませんでした")

    # 最初の輪郭を使用
    contour = contours[0]

    # 4. 輪郭の点を2つ選択 (間隔を point2_offset で調整)
    if len(contour) < 2:
        raise ValueError("輪郭の点が不足しています")

    point1 = contour[point1_index][0]
    point2_index = (point1_index + point2_offset) % len(contour)  # 周回計算で範囲外アクセスを防ぐ
    point2 = contour[point2_index][0]

    # 5. 距離を計算して角度を取得
    dx = point2[0] - point1[0]
    dy = point2[1] - point1[1]
    angle_radians = math.atan2(dy, dx)  # ラジアン単位
    angle_degrees = math.degrees(angle_radians)  # 度単位に変換

    print(f"Point1: {point1}, Point2: {point2}")
    print(f"dx: {dx}, dy: {dy}")
    print(f"Angle (radians): {angle_radians}, Angle (degrees): {angle_degrees}")

    # 6. 結果を画像に描画して表示
    img_color = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)  # 表示用カラー画像
    cv2.line(img_color, tuple(point1), tuple(point2), (0, 255, 0), 2)  # 線を描画
    cv2.circle(img_color, tuple(point1), 5, (255, 0, 0), -1)  # Point1に青い円を描画
    cv2.circle(img_color, tuple(point2), 5, (0, 0, 255), -1)  # Point2に赤い円を描画
    text_position = (10, 30)  # 角度表示位置
    cv2.putText(
        img_color,
        f"Angle: {angle_degrees:.2f} degrees",
        text_position,
        cv2.FONT_HERSHEY_SIMPLEX,
        0.8,
        (0, 255, 255),
        2,
        cv2.LINE_AA,
    )

    # 画像を表示
    cv2.imshow("Contour Angle", img_color)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return angle_radians, angle_degrees

# 画像パスを指定
image_path = "images/background/wakuta_mask.png"  # 画像パスを適宜変更

# 関数を呼び出して角度を取得
try:
    angle_radians, angle_degrees = calculate_angle_from_contour(image_path, point1_index=0, point2_offset=20)
except Exception as e:
    print(e)


Point1: [370 123], Point2: [330 133]
dx: -40, dy: 10
Angle (radians): 2.896613990462929, Angle (degrees): 165.96375653207352


In [33]:
import cv2
import numpy as np
import math

# 角度を取得する関数
def calculate_angle(point1, point2):
    """2点間の角度を計算する関数"""
    delta_x = point2[0] - point1[0]
    delta_y = point2[1] - point1[1]
    angle_radians = math.atan2(delta_y, delta_x)
    angle_degrees = math.degrees(angle_radians)
    return angle_degrees

# 昆虫画像を線分に沿って配置する関数
def place_insect_along_line(back_img, start_point, end_point, angle, insect_img):
    """指定した線分に昆虫画像を配置する"""
    # 昆虫画像の中心を回転の基準に設定
    insect_center = (insect_img.shape[1] // 2, insect_img.shape[0] // 2)
    
    # 昆虫画像を回転させる
    rotation_matrix = cv2.getRotationMatrix2D(insect_center, angle, 1.0)
    rotated_insect = cv2.warpAffine(insect_img, rotation_matrix, 
                                    (insect_img.shape[1], insect_img.shape[0]), 
                                    flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))

    # 配置位置の計算（線分の中点に配置）
    midpoint_x = (start_point[0] + end_point[0]) // 2
    midpoint_y = (start_point[1] + end_point[1]) // 2

    y1 = max(0, midpoint_y - insect_img.shape[0] // 2)
    y2 = min(back_img.shape[0], y1 + rotated_insect.shape[0])
    x1 = max(0, midpoint_x - insect_img.shape[1] // 2)
    x2 = min(back_img.shape[1], x1 + rotated_insect.shape[1])
    
    alpha_insect = rotated_insect[:, :, 3] / 255.0  # αチャンネル
    alpha_back = 1.0 - alpha_insect

    for c in range(0, 3):
        back_img[y1:y2, x1:x2, c] = (
            alpha_insect * rotated_insect[:, :, c] + alpha_back * back_img[y1:y2, x1:x2, c]
        )
    return back_img

# メイン処理
def main():
    # 背景画像（マスク）の読み込み
    mask_img = cv2.imread("images/background/wakuta_mask.png")
    copied_img = cv2.imread("images/background/wakuta_mask.png", cv2.IMREAD_UNCHANGED)
    insect_img = cv2.imread("images/wakuta_c/mantis8_AI.png", cv2.IMREAD_UNCHANGED)

    # 輪郭検出
    gray_mask = cv2.cvtColor(mask_img, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(gray_mask, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    points_outer = contours[0]

    # 輪郭に沿って昆虫画像を配置
    for i in range(0, len(points_outer)):
        insect_start_point = [points_outer[i][0][0], points_outer[i][0][1]]
        j = (i + 1) if (i + 1) != len(points_outer) else 0
        insect_end_point = [points_outer[j][0][0], points_outer[j][0][1]]

        # 線分の角度を計算
        angle = calculate_angle(insect_start_point, insect_end_point)
        print(f"Segment {i}: Start={insect_start_point}, End={insect_end_point}, Angle={angle:.2f}°")

        # 昆虫画像を配置
        copied_img = place_insect_along_line(copied_img, insect_start_point, insect_end_point, angle, insect_img)

    # 出力画像を保存
    cv2.imwrite("results/result_with_insects.png", copied_img)
    cv2.imshow("Result", copied_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Segment 0: Start=[370, 123], End=[369, 124], Angle=135.00°
Segment 1: Start=[369, 124], End=[363, 124], Angle=180.00°
Segment 2: Start=[363, 124], End=[362, 125], Angle=135.00°
Segment 3: Start=[362, 125], End=[356, 125], Angle=180.00°
Segment 4: Start=[356, 125], End=[355, 126], Angle=135.00°
Segment 5: Start=[355, 126], End=[351, 126], Angle=180.00°
Segment 6: Start=[351, 126], End=[350, 127], Angle=135.00°
Segment 7: Start=[350, 127], End=[345, 127], Angle=180.00°
Segment 8: Start=[345, 127], End=[344, 128], Angle=135.00°
Segment 9: Start=[344, 128], End=[341, 128], Angle=180.00°
Segment 10: Start=[341, 128], End=[340, 129], Angle=135.00°
Segment 11: Start=[340, 129], End=[338, 129], Angle=180.00°
Segment 12: Start=[338, 129], End=[337, 130], Angle=135.00°
Segment 13: Start=[337, 130], End=[336, 130], Angle=180.00°
Segment 14: Start=[336, 130], End=[335, 131], Angle=135.00°
Segment 15: Start=[335, 131], End=[334, 131], Angle=180.00°
Segment 16: Start=[334, 131], End=[333, 132], Angl

ValueError: operands could not be broadcast together with shapes (256,256) (255,256) 